## CPSC 439 (Spring 2022) Project 1: Turing Machines

###  Group Members
* Bradley Diep
* John Dinh
* Jason Duong
* Omid Nikjoo

### Documentation
   * Each transition is represented as a tuple $(a, b, c)$ encoded as $1^a01^b01^c0$ for $a, b, c \in N$
   * Each final state, $d$, is represented as $1^d0$ for $d\in N$
   * The $00$ token string deliminates objects T and S

### Example
``10110111001101 = T: (1, 11, 111), S: (11, 1) = T: (q0, q1, q2), S: (q1, q0)``

In [1]:
from collections import namedtuple

# Computational Model: DFA
DFA = namedtuple('DFA', ['T', 'S'])
BLANK, ZERO, ONE = ['.', '0', '1']

def encode(M):
    '''converts DFA model to binary representation'''
    code = ''
    for (current, symbol), successor in M.T.items():
        code += ''.join(list(map(lambda offset: ONE * (offset + 1) + ZERO,
                                 [current, symbol, successor])))
    
    code += ZERO + ''.join(list(map(lambda offset: ONE * (offset + 1), M.S)))
    return code

# DFA Representation
DFA.__repr__ = encode

In [2]:
'''Figure 6.3'''

# States
q0, q1, q2, q3 = list(range(4))

# Transition Function
δ = dict([ 
    ((q0, 0), q0), ((q0, 1), q1),
    ((q1, 0), q1), ((q1, 1), q0),
])

XOR_DFA = DFA(δ, [q1])
print(f'XOR-DFA: {XOR_DFA!r}')

XOR-DFA: 101010101101101101011011011010011


In [3]:
'''Figure 6.4'''

# Transition Function
δ = dict([
    ((q0, 0), q2), ((q0, 1), q3),
    ((q1, 0), q0), ((q1, 1), q3),
    ((q2, 0), q3), ((q2, 1), q1),
    ((q3, 0), q3), ((q3, 1), q3),
])

F_DFA = DFA(δ, [q0])
print(f'F-DFA: {F_DFA!r}')

F-DFA: 101011101011011110110101011011011110111010111101110110110111101011110111101101111001


### Turing Machine
* Run in https://turingmachinesimulator.com/ for it to work 
* Or click https://turingmachinesimulator.com/shared/nzyytbrnor to go directly there

name: Project 1 TM <br>
init: q0 <br>
accept: q0, q6 <br>

// initial state, separates depending on input

q0,0 <br>
q1,0,> <br>

q0,1 <br>
q5,1,> <br>

// satisfies Figure 6.4

q1,0 <br>
q3,0,> <br>

q1,1 <br>
q4,1,> <br>

q2,0 <br>
q1,0,> <br>

q2,1 <br>
q4,1,> <br>

q3,0 <br>
q4,0,> <br>

q3,1 <br>
q2,1,> <br>

q4,0 <br>
q4,0,> <br>

q4,1 <br>
q4,1,> <br>

q1,_ <br>
q1,_,- <br>

// satisfies Figure 6.3

q5,0 <br>
q5,0,> <br>

q5,1 <br>
q6,1,> <br>

q6,0 <br>
q6,0,> <br>

q6,1 <br>
q5,1,> <br>

q6,_ <br>
q6,_,- <br>